## 初期設定

### ライブラリのimport

In [1]:
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from pathlib import Path

from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import time
import icecream as ic
import os
from symusic import Score
import datetime

### deviceの設定

In [2]:
#device setting

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.cuda.is_available()

True

## データセットとトークナイザーの設定

### tokenizrの設定

In [3]:
# Creating a multitrack tokenizer, read the doc to explore all the parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\miditok\tokenizations\remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


### Mac環境

In [4]:

# Train the tokenizer with Byte Pair Encoding (BPE)
#files_paths = list(Path("../datasets").glob("**/*.mid"))
dataset_dir = Path("/Users/hapticslab/Programming/humusic/datasets/orchestra")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/orchestra")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/piano1_only")
files_paths = list(dataset_dir.glob("**/*.mid"))

tokenizer.train(vocab_size=30000, files_paths=files_paths)

#tokenizer.save(Path("path", "to", "save", "tokenizer.json"))
'''
tokenizer.save(Path("/Users/hapticslab/Programming/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
'''
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
#tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Split MIDIs into smaller chunks for training
dataset_chunks_dir = Path("/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/orchestra")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/orchestra")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/piano1_only")



### Windows環境

In [5]:

# Train the tokenizer with Byte Pair Encoding (BPE)
#files_paths = list(Path("../datasets").glob("**/*.mid"))
#dataset_dir = Path("/Users/hapticslab/Programming/humusic/datasets/piano1_only")
#dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/orchestra")
dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/piano1_only")
files_paths = list(dataset_dir.glob("*.mid"))

tokenizer.train(vocab_size=30000, files_paths=files_paths)


#tokenizer.save(Path("path", "to", "save", "tokenizer.json"))

#tokenizer.save(Path("/Users/hapticslab/Programming/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))

#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
#tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Split MIDIs into smaller chunks for training
#dataset_chunks_dir = Path("/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk/piano1_only")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/orchestra")
dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/piano1_only")



### トークンをDataloaderに保存

In [6]:

split_files_for_training(
    files_paths=files_paths,
    tokenizer=tokenizer,
    save_dir=dataset_chunks_dir,
    max_seq_len=1024,
)

# Create a Dataset, a DataLoader and a collator to train a model
dataset = DatasetMIDI(
    files_paths=list(dataset_chunks_dir.glob("**/*.mid")),
    tokenizer=tokenizer,
    max_seq_len=1024,
)
collator = DataCollator(tokenizer.pad_token_id, copy_inputs_as_labels=True)
dataloader = DataLoader(dataset, batch_size=16, collate_fn=collator)

Splitting music files (C:\Users\keisu\Programming\humor\humusic\midi_generator\midi_chunk\piano1_only): 100%|██████████| 283/283 [00:00<00:00, 718.22it/s]


## Attention付きLSTMによる学習，生成

学習パラメータの設定

In [7]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# LSMT with Attention
from LSTMwithAtt import LSTMwithAtt


hidden_size = 512
model = LSTMwithAtt(tokenizer, hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

num_epoch = 30

model = model.to(device)


In [33]:
from torch.nn.utils.rnn import pad_sequence

model.train()
start_time = time.time()

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    i = 0
    loss  = 0
    

    for batch in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epoch} |', unit='batch'):
        encoder_input = batch["input_ids"][:,1:].to(device)
        decoder_input = batch["input_ids"][:,:-1].to(device)
        #labels = batch["labels"][:,1:]
        
        e_input = pad_sequence(encoder_input, batch_first=True).to(device)
        d_input = pad_sequence(decoder_input, batch_first=True).to(device)
        ans = pad_sequence(labels, batch_first=True, padding_value=-100).to(device)
        
        #print(encoder_input.shape, decoder_input.shape, e_input.shape, d_input.shape)
        
        
        labels = batch["input_ids"][:,1:].to(device)    # 正解ラベル
        labels_one_hot = F.one_hot(labels, num_classes=tokenizer.vocab_size).float()
        
        #print(encoder_input.shape, decoder_input.shape)
        
        #out = model(e_input, d_input)
        out = model(encoder_input, decoder_input)
        
        
        #loss = criterion(out.view(-1, out.size(-1)), inputs.view(-1))
        #print(out.shape,labels_one_hot.shape)
        loss = criterion(out[0],labels_one_hot[0])
        #loss = criterion(out[0],labels[0])
        for h in range(1,len(out)):
            loss += criterion(out[h],labels_one_hot[h])
        #print(f'| epoch {epoch:3d} | {i:5d}/{len(dataloader)} batches | loss {loss.item():5.2f} | time {time.time()-epoch_start_time:5.2f}s')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    
    print(f'epoch {epoch+1}/{num_epoch} | loss {loss} | time {time.time()-epoch_start_time}')
    outfile = "models/" + "ltest"+ str(epoch+1) + ".model"
    torch.save(model.state_dict(),outfile)  
    

Epoch 1/30 Loss 0: 100%|██████████| 29/29 [00:50<00:00,  1.73s/batch]


epoch 1/30 loss : 79.83838653564453 time : 50.16966104507446


Epoch 2/30 Loss 0: 100%|██████████| 29/29 [00:57<00:00,  1.97s/batch]


epoch 2/30 loss : 77.52143096923828 time : 57.09922742843628


Epoch 3/30 Loss 0: 100%|██████████| 29/29 [00:57<00:00,  1.98s/batch]


...

epoch 28/30 loss : 48.733760833740234 time : 55.732603788375854


Epoch 29/30 Loss 0: 100%|██████████| 29/29 [00:58<00:00,  2.00s/batch]


epoch 29/30 loss : 39.6092414855957 time : 58.097989082336426


Epoch 30/30 Loss 0: 100%|██████████| 29/29 [00:58<00:00,  2.02s/batch]

epoch 30/30 loss : 31.659929275512695 time : 58.57815861701965


### 生成-プロンプトのトークン化

In [72]:
# 入力トークン列の例
prompt = ("prompt.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids.shape, input_ids

(torch.Size([1, 17]),
 tensor([[  445,   767,   234, 10777,  2616,  2793,  2314,  2008,  1360,  1855,
           1261,  2149,  1538,  4742,    58,  3483,  1005]], device='cuda:0'))

生成

In [73]:
# Generate a MIDI file

#model.load_state_dict(torch.load("models/lstmwithatttest30.model"))
model.load_state_dict(torch.load("models/ltest30.model"))

#torch.set_printoptions(threshold=torch.inf)
model.eval()

gen_token = []

with torch.no_grad():
    #prompt_input = torch.LongTensor( input_ids ).to(device)
    #print(input_ids.shape)
    x = model.input_emb(input_ids)
    #print(x.shape)
    ox, (hnx, cnx) = model.lstm1(x)
    #print(ox.shape, hnx.shape,cnx.shape)
    hnx, cnx = hnx[:,0,:], cnx[:,0,:]
    
    wid = input_ids[0][0]
    
    sl = 0
    while True:
        wids = torch.LongTensor([wid]).to(device)
        y = model.answer_emb(wids)
        #print(y.shape)
        
        oy, (hnx, cnx) = model.lstm2(y, (hnx, cnx))
        oy = oy.unsqueeze(1)
        ox1 = ox.permute(0,2,1)
        sim = torch.bmm(oy,ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = model.Wc(oy1)
        oy3 = model.W(oy2)
        wid = torch.argmax(oy3[0]).item()
        gen_token.append(wid)
        #if (wid == esid):
            #break
        if (sl == 2000):
            break
        sl += 1
    print(gen_token)

[6877, 806, 20712, 8903, 356, 4538, 1735, 6327, 27082, 1727, 3876, 17888, 4489, 6327, 17098, 11337, 5718, 1735, 26813, 13134, 1727, 26823, 27082, 11514, 600, 806, 17098, 600, 8018, 279, 600, 600, 1788, 14379, 258, 11514, 600, 806, 17098, 600, 8018, 746, 506, 20871, 506, 20871, 600, 6398, 17888, 14854, 17098, 7027, 18233, 7027, 18219, 5741, 7027, 17888, 17888, 1238, 17888, 17888, 9896, 26813, 4695, 600, 1409, 17888, 9946, 5718, 11337, 600, 5718, 746, 20871, 27369, 600, 600, 600, 20596, 600, 600, 600, 7575, 2395, 258, 11514, 17098, 17039, 12279, 29709, 11337, 982, 691, 5741, 2555, 11514, 12799, 11514, 27369, 21632, 27688, 17973, 26619, 11514, 11514, 26050, 9273, 28241, 20871, 29709, 600, 600, 15096, 806, 806, 1029, 806, 23004, 27688, 12799, 11514, 11514, 17973, 511, 284, 20871, 18219, 6935, 5718, 17973, 9131, 1660, 746, 825, 11514, 600, 806, 600, 8018, 10102, 12799, 5322, 12799, 11514, 11514, 11514, 1788, 14828, 17973, 6398, 4989, 17888, 17888, 1760, 17888, 806, 600, 600, 600, 600, 600, 

In [74]:
generated = tokenizer.decode(gen_token)
print("Generated MIDI Tokens:", generated)

generated.dump_midi("generated_test10.mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=1234, tracks=1, notes=1352, time_sig=1, key_sig=0, markers=0)


## Trasformerによる学習，生成

### 学習パラメータの設定

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel


config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=2048,
    n_ctx=2048,
    n_embd=256,
    n_layer=8,
    n_head=8,
    
    #bos_token_id=tokenizer["BOS_None"],
    #eos_token_id=tokenizer["EOS_None"],
    pad_token_id=tokenizer.pad_token_id
    
    )

model = GPT2LMHeadModel(config).to(device)

# 損失関数とオプティマイザ
criterion = torch.nn.CrossEntropyLoss()  # トークンの予測タスクに使う
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)  # 適切な学習率を設定

### 学習

In [13]:


# 学習ループ
epochs = 2 # エポック数
for epoch in range(epochs):
    model.train()  # モデルを学習モードに切り替え
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs = batch["input_ids"].to(device)  # トークンID列
        labels = batch["labels"].to(device)    # 正解ラベル
        attention_mask = batch["attention_mask"].to(device)  # マスク（任意）

        # 勾配を初期化
        optimizer.zero_grad()

        # モデルの前方計算
        outputs = model(
            input_ids=inputs,
            labels=labels,
            attention_mask=attention_mask,
        )
        loss = outputs.loss  # GPT2LMHeadModelは自動で損失を計算する

        # 勾配の計算とパラメータの更新
        loss.backward()
        optimizer.step()

        # 損失を記録
        total_loss += loss.item()

    # エポックごとの平均損失を出力
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    model.save_pretrained("models/piano1_only/test" + str(epoch+1) + ".pt")

Epoch 1/2: 100%|██████████| 29/29 [02:33<00:00,  5.30s/it]


Epoch 1/2 - Loss: 10.4803


Epoch 2/2: 100%|██████████| 29/29 [02:26<00:00,  5.04s/it]

Epoch 2/2 - Loss: 9.8964


### 生成-プロンプトのトークン化

In [18]:


# 入力トークン列の例
prompt = ("prompt.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids

tensor([[  445,   767,   234, 10777,  2616,  2793,  2314,  2008,  1360,  1855,
          1261,  2149,  1538,  4742,    58,  3483,  1005]], device='cuda:0')

### 生成-メインプロセス

In [15]:
#regenerate

# モデルの読み込み

model = GPT2LMHeadModel.from_pretrained("models/piano1_only/test2.pt")
model.to(device)
model.eval()

# モデルによる生成
# output = model.generate(
#     input_ids=input_ids,
#     max_length=1024,
#     num_beams=5,
#     no_repeat_ngram_size=2,
# )

# モデルによる生成
output = model.generate(
    input_ids=input_ids,
    max_length=1024,
    num_beams=5,  # ビームサーチ
    no_repeat_ngram_size=2,  # 同じn-gramを繰り返さない
    temperature=2.0,  # サンプリングの多様性を調整
    top_k=200,  # 上位k個のトークンをサンプリング
    top_p=0.90,  # 累積確率cutoff
    
)

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `200` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


### 生成-結果をdetokenize

In [16]:
# トークン列をデコードして結果を表示
generated = tokenizer.decode(output[0].tolist())
print("Generated MIDI Tokens:", generated)

generated.dump_midi("generated_test5.mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=5480, tracks=1, notes=47, time_sig=1, key_sig=0, markers=0)


## python(GPU)のキャッシュクリア

In [75]:
#del model, dataset, dataloader, collator
torch.cuda.empty_cache()

## 生成したmidiの再生
強制終了するとipykernelがクラッシュするので注意

In [76]:
#pygameによる再生

file_name = Path("generated_test10.mid")

import time
import pygame


'''
# Play the MIDI file in the current directory
for msg in mid.play():
    time.sleep(msg.time)
    if not msg.is_meta:
        print(msg)
'''
# Initialize pygame
pygame.init()

# Set up the mixer
pygame.mixer.init()

# Load the MIDI file
#pygame.mixer.music.load('bach_850.mid')
pygame.mixer.music.load(file_name)
#pygame.mixer.music.load('test.mid')

# Play the MIDI file
pygame.mixer.music.set_volume(0.25)
pygame.mixer.music.play()


# Keep the program running until the music stops
while pygame.mixer.music.get_busy():
    if input("Press 'q' to quit: ") == 'q':
        break
    pygame.time.Clock().tick(10)

# Quit pygame
pygame.quit()

# prompt.midの作成セル

In [31]:
# まずは1小節のnote（ド、C4）を1つ生成してみる
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(120))) # bpm120


track.append(Message('note_on', note=60, velocity=100, time=0))
track.append(Message('note_off', note=60, velocity=100, time=480))
track.append(Message('note_on', note=74, velocity=57, time=120))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=60, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=81, velocity=62, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_off', note=81, velocity=0, time=100))
track.append(Message('note_on', note=74, velocity=58, time=0))
track.append(Message('note_on', note=76, velocity=58, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=58, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=61, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=79, velocity=60, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))


mid.save('prompt.mid') # MidiFileを保存
